In [25]:
import pandas as pd

In [27]:
amazon = pd.read_csv('amazon_prime_titles.csv')

In [29]:
amazon.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


In [31]:
amazon.isnull().sum()

show_id            0
type               0
title              0
director        2083
cast            1233
country         8996
date_added      9513
release_year       0
rating           337
duration           0
listed_in          0
description        0
dtype: int64

MISSING VALUES TREATMENT

In [33]:
amazon['director']=amazon['director'].fillna('Unknown')
amazon['cast']=amazon['cast'].fillna('Unknown')

In [23]:
amazon.isnull().sum()

show_id            0
type               0
title              0
director           0
cast               0
country         8996
date_added      9513
release_year       0
rating           337
duration           0
listed_in          0
description        0
dtype: int64

EXPLODING CLOUMN AS MUTIPLE ENTRIES IN ONE COLUMN (COUNTRY,GENRE)

In [35]:
exploded = amazon.assign(country=amazon['country'].str.split(',')).explode('country')

In [41]:
exploded = amazon.assign(listed_in=amazon['listed_in'].str.split(',')).explode('listed_in')

In [43]:
exploded.isnull().sum()

show_id             0
type                0
title               0
director            0
cast                0
country         16931
date_added      17989
release_year        0
rating            750
duration            0
listed_in           0
description         0
dtype: int64

In [ ]:
MISSING VALUE TREAMENT FOR EXPLODED DATAFRAME

In [45]:
country_filled = exploded.groupby('listed_in')['country'].apply(
    lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else 'Unknown')
).reset_index(drop=True)

exploded['country'] = country_filled

In [47]:
exploded.isnull().sum()

show_id             0
type                0
title               0
director            0
cast                0
country             0
date_added      17989
release_year        0
rating            750
duration            0
listed_in           0
description         0
dtype: int64

In [49]:
exploded['date_added'] = exploded.groupby('listed_in')['date_added'].apply(
    lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else "Unknown")
).reset_index(drop=True)

In [51]:
exploded['rating'] = exploded.groupby('listed_in')['rating'].apply(
    lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else "Unknown")
).reset_index(drop=True)

In [53]:
exploded.isnull().sum()

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

In [ ]:
MAKING UNIQUE COUNTRIES AND REMOVING COMMA AGAIN

In [57]:
exploded['country'] = exploded['country'].str.split(', ')
exploded = exploded.explode('country')

In [59]:
exploded['country'].unique()

array(['United States', 'India', 'Brazil', 'Canada', 'France', 'Denmark',
       'United Kingdom', 'Ireland', 'Belgium', 'Luxembourg', 'Japan',
       'Singapore', 'South Korea', 'Mexico', 'Germany', 'Hong Kong',
       'Unknown', 'China', 'Austria', 'Italy', 'Switzerland', 'Chile',
       'Argentina', 'Spain', 'Australia', 'Sweden', 'Norway', 'Hungary',
       'Poland', 'Netherlands', 'Egypt', 'Portugal', 'South Africa',
       'New Zealand', 'Thailand', 'United Arab Emirates', 'Malaysia',
       'Monaco', 'Colombia'], dtype=object)

In [63]:
exploded['listed_in'].unique()

array(['Comedy', ' Drama', 'Drama', ' International', 'Action',
       ' Suspense', 'Documentary', ' Fantasy', 'Fantasy', ' Kids',
       ' Special Interest', ' Science Fiction', 'Adventure', 'Horror',
       ' Sports', 'Kids', ' Talk Show and Variety', 'Science Fiction',
       ' Anime', ' Comedy', 'Arts', ' Entertainment', ' and Culture',
       'TV Shows', 'Animation', ' Adventure', ' Animation', 'Anime',
       'Music Videos and Concerts', 'Fitness', 'Faith and Spirituality',
       'Special Interest', ' Military and War', ' Western', 'Suspense',
       ' LGBTQ', ' Romance', ' Unscripted', 'Unscripted', ' Documentary',
       'Western', ' Faith and Spirituality', ' Young Adult Audience',
       'Arthouse', 'Sports', 'Military and War', ' Horror',
       'International', ' Music Videos and Concerts', ' Historical',
       'Romance', ' Arts', 'Young Adult Audience',
       'Talk Show and Variety', 'LGBTQ', ' Arthouse', 'Historical'],
      dtype=object)

In [ ]:
REMOVING COMMA AS SOME HAVE STILL

In [69]:
exploded['listed_in'] = exploded['listed_in'].str.strip()
exploded['listed_in'].unique()

array(['Comedy', 'Drama', 'International', 'Action', 'Suspense',
       'Documentary', 'Fantasy', 'Kids', 'Special Interest',
       'Science Fiction', 'Adventure', 'Horror', 'Sports',
       'Talk Show and Variety', 'Anime', 'Arts', 'Entertainment',
       'and Culture', 'TV Shows', 'Animation',
       'Music Videos and Concerts', 'Fitness', 'Faith and Spirituality',
       'Military and War', 'Western', 'LGBTQ', 'Romance', 'Unscripted',
       'Young Adult Audience', 'Arthouse', 'Historical'], dtype=object)

In [61]:
exploded['rating'].unique()

array(['TV-Y7', '13+', 'TV-14', 'R', 'NR', '7+', 'TV-PG', '18+', 'TV-NR',
       'ALL', 'PG-13', 'PG', '16+', 'UNRATED', 'TV-MA', 'G', 'TV-G',
       'TV-Y', 'NC-17', 'AGES_18_', 'NOT_RATE', 'AGES_16_'], dtype=object)

In [73]:
exploded.to_csv('amazon_formatted.csv',index=False)